In [1]:
# Import Dependencies
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [2]:
# Import DataFrame 
main_df = pd.read_csv('Rates_MO.csv')
main_df.columns

Index(['Unnamed: 0', 'observation_date', 'C&I_DELNQ', 'CCARD_CO',
       'CCARD_DELNQ', 'CORP_DEBT_NET_WORTH', 'CORP_SAVINGS_LEVEL', 'CRE_CO',
       'CRE_DELNQ', 'GDP', 'Homeowner_Vacancy_rate', 'Household_DBT_Inc',
       'Mortgage_CO', 'Mortgage_DELNQ', 'Rental_Vacancy_Rate',
       'Consumer_Confidence', 'FEDFUNDS', 'Manufacturing_Confidence',
       'SAVINGS_RATE_MO', 'UNRATE', 'C&I_CO'],
      dtype='object')

# Credit Cards Group

In [3]:
# Create credit card dataset
ccard_df = main_df[['observation_date', 'CCARD_CO', 'CCARD_DELNQ', 'GDP', 'Household_DBT_Inc', 'Consumer_Confidence', 'FEDFUNDS', 'SAVINGS_RATE_MO', 'UNRATE']]
 
# Create copy for bins
ccard_bin_df = ccard_df.copy()

In [4]:
# Binning the data for classification Question: 
ccard_bin_df["CCARD_CO_BIN"] = pd.qcut(ccard_df['CCARD_CO'],4, labels= [1, 2, 3, 4])

# Seperate the y and X variables
y = ccard_bin_df["CCARD_CO_BIN"]

In [5]:
ccard_df = ccard_df.drop(columns=["CCARD_CO","observation_date"])
ccard_df

,CCARD_DELNQ,GDP,Household_DBT_Inc,Consumer_Confidence,FEDFUNDS,SAVINGS_RATE_MO,UNRATE
0,5.26,-1.9,11.578032,66.8,6.91,9.4,6.4
1,5.26,-1.9,11.578032,70.4,6.25,9.0,6.6
2,5.26,-1.9,11.578032,87.7,6.12,8.1,6.8
3,5.48,3.2,11.434237,81.8,5.91,8.7,6.7
4,5.48,3.2,11.434237,78.3,5.78,8.5,6.9
...,...,...,...,...,...,...,...
385,2.43,2.2,9.848832,67.0,4.57,4.7,3.6
386,2.43,2.2,9.848832,62.0,4.65,5.2,3.5
387,2.77,2.1,9.826692,63.5,4.83,5.2,3.4
388,2.77,2.1,9.826692,59.2,5.06,5.3,3.7


# Deep Learning - Credit Cards

In [6]:
## Splitting into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(ccard_df, y, random_state=78)

In [7]:
# Creating StandardScaler instance
scaler = MinMaxScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
print(X_train_scaled)

[[0.16858238 0.48089172 0.31664056 ... 0.65889213 0.10784314 0.01769912]
 [0.5210728  0.55414013 0.60881321 ... 0.75364431 0.13398693 0.19469027]
 [0.58429119 0.45700637 0.43389017 ... 0.44023324 0.24183007 0.31858407]
 ...
 [0.57471264 0.55254777 0.66558862 ... 0.75072886 0.08496732 0.0619469 ]
 [0.33333333 0.53343949 0.44074702 ... 0.57725948 0.19934641 0.23893805]
 [0.47318008 0.48566879 0.95252878 ... 0.75801749 0.04248366 0.10619469]]


In [9]:
print("X_train set shape: ", X_train_scaled.shape)
print("y_train set shape: ", y_train.shape)

X_train set shape:  (292, 7)
y_train set shape:  (292,)


In [10]:
y_train.values

[1, 3, 2, 2, 1, ..., 2, 1, 3, 1, 2]
Length: 292
Categories (4, int64): [1 < 2 < 3 < 4]

In [11]:
num_classes = 5
y_train = to_categorical(y_train.values, num_classes)
y_test = to_categorical(y_test.values, num_classes)
y_train[0]

array([0., 1., 0., 0., 0.], dtype=float32)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
keras.backend.clear_session()
model = Sequential()

number_input_features = X_train_scaled.shape[1]
hidden_nodes_layers1 = 60
hidden_nodes_layers2 = 60
hidden_nodes_layers3 = 60
hidden_nodes_layers4 = 40


model.add(Dense(20, activation='relu', input_dim=number_input_features))
model.add(Dense(hidden_nodes_layers1, activation='relu'))
model.add(Dense(hidden_nodes_layers2, activation='relu'))
model.add(Dense(hidden_nodes_layers3, activation='relu'))
model.add(Dense(hidden_nodes_layers4, activation='relu'))
# model.add(Dense(hidden_nodes_layers5, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                160       
                                                                 
 dense_1 (Dense)             (None, 60)                1260      
                                                                 
 dense_2 (Dense)             (None, 60)                3660      
                                                                 
 dense_3 (Dense)             (None, 60)                3660      
                                                                 
 dense_4 (Dense)             (None, 40)                2440      
                                                                 
 dense_5 (Dense)             (None, 5)                 205       
                                                                 
Total params: 11385 (44.47 KB)
Trainable params: 11385 (

In [13]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=200,
    shuffle=True,
    verbose=2
)

Epoch 1/200
10/10 - 0s - loss: 1.5721 - accuracy: 0.4349 - 158ms/epoch - 16ms/step
Epoch 2/200
10/10 - 0s - loss: 1.5060 - accuracy: 0.4452 - 5ms/epoch - 470us/step
Epoch 3/200
10/10 - 0s - loss: 1.4312 - accuracy: 0.3082 - 4ms/epoch - 440us/step
Epoch 4/200
10/10 - 0s - loss: 1.3320 - accuracy: 0.4212 - 4ms/epoch - 442us/step
Epoch 5/200
10/10 - 0s - loss: 1.2238 - accuracy: 0.5000 - 6ms/epoch - 573us/step
Epoch 6/200
10/10 - 0s - loss: 1.1070 - accuracy: 0.6438 - 5ms/epoch - 497us/step
Epoch 7/200
10/10 - 0s - loss: 0.9886 - accuracy: 0.5959 - 5ms/epoch - 498us/step
Epoch 8/200
10/10 - 0s - loss: 0.9081 - accuracy: 0.6644 - 5ms/epoch - 502us/step
Epoch 9/200
10/10 - 0s - loss: 0.8424 - accuracy: 0.6164 - 5ms/epoch - 537us/step
Epoch 10/200
10/10 - 0s - loss: 0.8036 - accuracy: 0.6712 - 5ms/epoch - 538us/step
Epoch 11/200
10/10 - 0s - loss: 0.7794 - accuracy: 0.6267 - 5ms/epoch - 526us/step
Epoch 12/200
10/10 - 0s - loss: 0.7546 - accuracy: 0.6404 - 6ms/epoch - 587us/step
Epoch 13/200

Epoch 100/200
10/10 - 0s - loss: 0.2911 - accuracy: 0.8904 - 5ms/epoch - 503us/step
Epoch 101/200
10/10 - 0s - loss: 0.2813 - accuracy: 0.8870 - 5ms/epoch - 502us/step
Epoch 102/200
10/10 - 0s - loss: 0.3113 - accuracy: 0.8527 - 5ms/epoch - 514us/step
Epoch 103/200
10/10 - 0s - loss: 0.2829 - accuracy: 0.8870 - 5ms/epoch - 516us/step
Epoch 104/200
10/10 - 0s - loss: 0.3144 - accuracy: 0.8767 - 5ms/epoch - 518us/step
Epoch 105/200
10/10 - 0s - loss: 0.3003 - accuracy: 0.8699 - 5ms/epoch - 532us/step
Epoch 106/200
10/10 - 0s - loss: 0.3710 - accuracy: 0.8459 - 5ms/epoch - 523us/step
Epoch 107/200
10/10 - 0s - loss: 0.3387 - accuracy: 0.8699 - 5ms/epoch - 516us/step
Epoch 108/200
10/10 - 0s - loss: 0.3081 - accuracy: 0.8733 - 5ms/epoch - 492us/step
Epoch 109/200
10/10 - 0s - loss: 0.3464 - accuracy: 0.8322 - 5ms/epoch - 510us/step
Epoch 110/200
10/10 - 0s - loss: 0.3336 - accuracy: 0.8562 - 6ms/epoch - 551us/step
Epoch 111/200
10/10 - 0s - loss: 0.3161 - accuracy: 0.8562 - 5ms/epoch - 520

Epoch 198/200
10/10 - 0s - loss: 0.1748 - accuracy: 0.9452 - 6ms/epoch - 628us/step
Epoch 199/200
10/10 - 0s - loss: 0.1982 - accuracy: 0.9178 - 6ms/epoch - 616us/step
Epoch 200/200
10/10 - 0s - loss: 0.2042 - accuracy: 0.9418 - 7ms/epoch - 687us/step


In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.3067 - accuracy: 0.9082 - 63ms/epoch - 16ms/step
Loss: 0.30674079060554504, Accuracy: 0.9081632494926453


# Mortgage Loan Group

In [15]:
# Create mortgage loan dataset
mort_df = main_df[['observation_date', 'Mortgage_CO', 'Mortgage_DELNQ', 'GDP', 'Household_DBT_Inc', 'Consumer_Confidence', 'FEDFUNDS', 'SAVINGS_RATE_MO', 'UNRATE']]

mort_bin_df = mort_df.copy()

mort_df.columns

Index(['observation_date', 'Mortgage_CO', 'Mortgage_DELNQ', 'GDP',
       'Household_DBT_Inc', 'Consumer_Confidence', 'FEDFUNDS',
       'SAVINGS_RATE_MO', 'UNRATE'],
      dtype='object')

In [16]:
# Binning the data for classification Question: 
mort_bin_df["Mortgage_CO_BIN"] = pd.qcut(mort_df['Mortgage_CO'],4, labels= [1, 2, 3, 4])
# Define the dependent Y variable
y = mort_bin_df["Mortgage_CO_BIN"]
mort_df = mort_df.drop(columns=['Mortgage_CO','observation_date'])
mort_df

,Mortgage_DELNQ,GDP,Household_DBT_Inc,Consumer_Confidence,FEDFUNDS,SAVINGS_RATE_MO,UNRATE
0,3.09,-1.9,11.578032,66.8,6.91,9.4,6.4
1,3.09,-1.9,11.578032,70.4,6.25,9.0,6.6
2,3.09,-1.9,11.578032,87.7,6.12,8.1,6.8
3,3.18,3.2,11.434237,81.8,5.91,8.7,6.7
4,3.18,3.2,11.434237,78.3,5.78,8.5,6.9
...,...,...,...,...,...,...,...
385,1.74,2.2,9.848832,67.0,4.57,4.7,3.6
386,1.74,2.2,9.848832,62.0,4.65,5.2,3.5
387,1.72,2.1,9.826692,63.5,4.83,5.2,3.4
388,1.72,2.1,9.826692,59.2,5.06,5.3,3.7


# Deep Learning: Mortgage Loans

In [17]:
## Splitting into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(ccard_df, y, random_state=78)

In [18]:
# Creating StandardScaler instance
scaler = MinMaxScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
print("X_train set shape: ", X_train_scaled.shape)
print("y_train set shape: ", y_train.shape)

X_train set shape:  (292, 7)
y_train set shape:  (292,)


In [20]:
y_train.values

[1, 2, 3, 3, 4, ..., 2, 1, 2, 3, 3]
Length: 292
Categories (4, int64): [1 < 2 < 3 < 4]

In [21]:
num_classes = 5
y_train = to_categorical(y_train.values, num_classes)
y_test = to_categorical(y_test.values, num_classes)
y_train[0]

array([0., 1., 0., 0., 0.], dtype=float32)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
keras.backend.clear_session()
model = Sequential()

number_input_features = X_train_scaled.shape[1]
hidden_nodes_layers1 = 60
hidden_nodes_layers2 = 60
hidden_nodes_layers3 = 60
hidden_nodes_layers4 = 40


model.add(Dense(20, activation='relu', input_dim=number_input_features))
model.add(Dense(hidden_nodes_layers1, activation='relu'))
model.add(Dense(hidden_nodes_layers2, activation='relu'))
model.add(Dense(hidden_nodes_layers3, activation='relu'))
model.add(Dense(hidden_nodes_layers4, activation='relu'))
# model.add(Dense(hidden_nodes_layers5, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                160       
                                                                 
 dense_1 (Dense)             (None, 60)                1260      
                                                                 
 dense_2 (Dense)             (None, 60)                3660      
                                                                 
 dense_3 (Dense)             (None, 60)                3660      
                                                                 
 dense_4 (Dense)             (None, 40)                2440      
                                                                 
 dense_5 (Dense)             (None, 5)                 205       
                                                                 
Total params: 11385 (44.47 KB)
Trainable params: 11385 (

In [23]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=200,
    shuffle=True,
    verbose=2
)

Epoch 1/200
10/10 - 0s - loss: 1.5899 - accuracy: 0.2877 - 133ms/epoch - 13ms/step
Epoch 2/200
10/10 - 0s - loss: 1.5493 - accuracy: 0.2808 - 6ms/epoch - 575us/step
Epoch 3/200
10/10 - 0s - loss: 1.4974 - accuracy: 0.2808 - 5ms/epoch - 460us/step
Epoch 4/200
10/10 - 0s - loss: 1.4274 - accuracy: 0.2842 - 5ms/epoch - 468us/step
Epoch 5/200
10/10 - 0s - loss: 1.3511 - accuracy: 0.3733 - 6ms/epoch - 558us/step
Epoch 6/200
10/10 - 0s - loss: 1.2602 - accuracy: 0.4829 - 5ms/epoch - 499us/step
Epoch 7/200
10/10 - 0s - loss: 1.1629 - accuracy: 0.5479 - 5ms/epoch - 475us/step
Epoch 8/200
10/10 - 0s - loss: 1.0401 - accuracy: 0.6164 - 6ms/epoch - 557us/step
Epoch 9/200
10/10 - 0s - loss: 0.9525 - accuracy: 0.6267 - 5ms/epoch - 488us/step
Epoch 10/200
10/10 - 0s - loss: 0.9264 - accuracy: 0.6062 - 5ms/epoch - 517us/step
Epoch 11/200
10/10 - 0s - loss: 0.8761 - accuracy: 0.6849 - 6ms/epoch - 584us/step
Epoch 12/200
10/10 - 0s - loss: 0.8502 - accuracy: 0.6473 - 5ms/epoch - 505us/step
Epoch 13/200

Epoch 100/200
10/10 - 0s - loss: 0.3076 - accuracy: 0.8699 - 5ms/epoch - 536us/step
Epoch 101/200
10/10 - 0s - loss: 0.2902 - accuracy: 0.8870 - 5ms/epoch - 527us/step
Epoch 102/200
10/10 - 0s - loss: 0.2620 - accuracy: 0.9247 - 5ms/epoch - 514us/step
Epoch 103/200
10/10 - 0s - loss: 0.2894 - accuracy: 0.9007 - 5ms/epoch - 513us/step
Epoch 104/200
10/10 - 0s - loss: 0.3818 - accuracy: 0.8390 - 5ms/epoch - 526us/step
Epoch 105/200
10/10 - 0s - loss: 0.4070 - accuracy: 0.8219 - 5ms/epoch - 526us/step
Epoch 106/200
10/10 - 0s - loss: 0.3181 - accuracy: 0.8870 - 5ms/epoch - 518us/step
Epoch 107/200
10/10 - 0s - loss: 0.2682 - accuracy: 0.9075 - 5ms/epoch - 511us/step
Epoch 108/200
10/10 - 0s - loss: 0.2748 - accuracy: 0.8904 - 5ms/epoch - 544us/step
Epoch 109/200
10/10 - 0s - loss: 0.2531 - accuracy: 0.9110 - 5ms/epoch - 525us/step
Epoch 110/200
10/10 - 0s - loss: 0.2586 - accuracy: 0.8904 - 5ms/epoch - 502us/step
Epoch 111/200
10/10 - 0s - loss: 0.2759 - accuracy: 0.8870 - 5ms/epoch - 537

Epoch 198/200
10/10 - 0s - loss: 0.1904 - accuracy: 0.9144 - 6ms/epoch - 556us/step
Epoch 199/200
10/10 - 0s - loss: 0.1722 - accuracy: 0.9247 - 5ms/epoch - 520us/step
Epoch 200/200
10/10 - 0s - loss: 0.1838 - accuracy: 0.9212 - 5ms/epoch - 515us/step


In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.3752 - accuracy: 0.8878 - 43ms/epoch - 11ms/step
Loss: 0.3752179443836212, Accuracy: 0.8877550959587097


# C&I Loan Group

In [25]:
# Create C&I loan dataset
CI_df = main_df[['observation_date', 'C&I_CO', 'C&I_DELNQ', 'GDP', 'CORP_DEBT_NET_WORTH', 'Manufacturing_Confidence', 'FEDFUNDS', 'CORP_SAVINGS_LEVEL', 'UNRATE']]

CI_bin_df = mort_df.copy()

CI_df.columns

Index(['observation_date', 'C&I_CO', 'C&I_DELNQ', 'GDP', 'CORP_DEBT_NET_WORTH',
       'Manufacturing_Confidence', 'FEDFUNDS', 'CORP_SAVINGS_LEVEL', 'UNRATE'],
      dtype='object')

In [26]:
# Binning the data for classification Question: 
CI_bin_df["C&I_CO_BIN"] = pd.qcut(CI_df['C&I_CO'],4, labels= [1, 2, 3, 4])

# Define the dependent Y variable
y = CI_bin_df["C&I_CO_BIN"]

CI_df = CI_df.drop(columns=['C&I_CO','observation_date'])
CI_df

,C&I_DELNQ,GDP,CORP_DEBT_NET_WORTH,Manufacturing_Confidence,FEDFUNDS,CORP_SAVINGS_LEVEL,UNRATE
0,6.29,-1.9,46.132964,96.951745,6.91,77.964,6.4
1,6.29,-1.9,46.132964,96.972896,6.25,77.964,6.6
2,6.29,-1.9,46.132964,97.223425,6.12,77.964,6.8
3,6.41,3.2,46.289579,97.678049,5.91,81.294,6.7
4,6.41,3.2,46.289579,98.292261,5.78,81.294,6.9
...,...,...,...,...,...,...,...
385,0.98,2.2,40.497128,98.951152,4.57,263.194,3.6
386,0.98,2.2,40.497128,98.913862,4.65,263.194,3.5
387,1.01,2.1,39.659559,98.904602,4.83,367.036,3.4
388,1.01,2.1,39.659559,98.887364,5.06,367.036,3.7


# Deep Learning: C&I Loans

In [27]:
## Splitting into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(ccard_df, y, random_state=78)

In [28]:
# Creating StandardScaler instance
scaler = MinMaxScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
print("X_train set shape: ", X_train_scaled.shape)
print("y_train set shape: ", y_train.shape)

X_train set shape:  (292, 7)
y_train set shape:  (292,)


In [30]:
y_train.values

[1, 2, 4, 2, 1, ..., 1, 2, 3, 2, 3]
Length: 292
Categories (4, int64): [1 < 2 < 3 < 4]

In [31]:
num_classes = 5
y_train = to_categorical(y_train.values, num_classes)
y_test = to_categorical(y_test.values, num_classes)
y_train[0]

array([0., 1., 0., 0., 0.], dtype=float32)

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
keras.backend.clear_session()
model = Sequential()

number_input_features = X_train_scaled.shape[1]
hidden_nodes_layers1 = 60
hidden_nodes_layers2 = 60
hidden_nodes_layers3 = 60
hidden_nodes_layers4 = 40


model.add(Dense(20, activation='relu', input_dim=number_input_features))
model.add(Dense(hidden_nodes_layers1, activation='relu'))
model.add(Dense(hidden_nodes_layers2, activation='relu'))
model.add(Dense(hidden_nodes_layers3, activation='relu'))
model.add(Dense(hidden_nodes_layers4, activation='relu'))
# model.add(Dense(hidden_nodes_layers5, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                160       
                                                                 
 dense_1 (Dense)             (None, 60)                1260      
                                                                 
 dense_2 (Dense)             (None, 60)                3660      
                                                                 
 dense_3 (Dense)             (None, 60)                3660      
                                                                 
 dense_4 (Dense)             (None, 40)                2440      
                                                                 
 dense_5 (Dense)             (None, 5)                 205       
                                                                 
Total params: 11385 (44.47 KB)
Trainable params: 11385 (

In [33]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=200,
    shuffle=True,
    verbose=2
)

Epoch 1/200
10/10 - 0s - loss: 1.5830 - accuracy: 0.3973 - 129ms/epoch - 13ms/step
Epoch 2/200
10/10 - 0s - loss: 1.5315 - accuracy: 0.3836 - 5ms/epoch - 512us/step
Epoch 3/200
10/10 - 0s - loss: 1.4629 - accuracy: 0.3733 - 5ms/epoch - 477us/step
Epoch 4/200
10/10 - 0s - loss: 1.3859 - accuracy: 0.4384 - 5ms/epoch - 454us/step
Epoch 5/200
10/10 - 0s - loss: 1.3005 - accuracy: 0.4555 - 4ms/epoch - 437us/step
Epoch 6/200
10/10 - 0s - loss: 1.2416 - accuracy: 0.4521 - 4ms/epoch - 438us/step
Epoch 7/200
10/10 - 0s - loss: 1.1630 - accuracy: 0.4726 - 5ms/epoch - 529us/step
Epoch 8/200
10/10 - 0s - loss: 1.1081 - accuracy: 0.5000 - 5ms/epoch - 485us/step
Epoch 9/200
10/10 - 0s - loss: 1.0679 - accuracy: 0.5171 - 5ms/epoch - 480us/step
Epoch 10/200
10/10 - 0s - loss: 1.0312 - accuracy: 0.5205 - 6ms/epoch - 592us/step
Epoch 11/200
10/10 - 0s - loss: 1.0710 - accuracy: 0.4932 - 5ms/epoch - 539us/step
Epoch 12/200
10/10 - 0s - loss: 0.9804 - accuracy: 0.6164 - 5ms/epoch - 529us/step
Epoch 13/200

Epoch 100/200
10/10 - 0s - loss: 0.5176 - accuracy: 0.7705 - 5ms/epoch - 520us/step
Epoch 101/200
10/10 - 0s - loss: 0.5309 - accuracy: 0.7637 - 6ms/epoch - 552us/step
Epoch 102/200
10/10 - 0s - loss: 0.5105 - accuracy: 0.7911 - 5ms/epoch - 533us/step
Epoch 103/200
10/10 - 0s - loss: 0.5180 - accuracy: 0.7637 - 5ms/epoch - 519us/step
Epoch 104/200
10/10 - 0s - loss: 0.5393 - accuracy: 0.7534 - 5ms/epoch - 515us/step
Epoch 105/200
10/10 - 0s - loss: 0.5163 - accuracy: 0.7637 - 5ms/epoch - 525us/step
Epoch 106/200
10/10 - 0s - loss: 0.5031 - accuracy: 0.7808 - 6ms/epoch - 555us/step
Epoch 107/200
10/10 - 0s - loss: 0.4895 - accuracy: 0.7877 - 5ms/epoch - 532us/step
Epoch 108/200
10/10 - 0s - loss: 0.5050 - accuracy: 0.7705 - 5ms/epoch - 520us/step
Epoch 109/200
10/10 - 0s - loss: 0.4974 - accuracy: 0.7808 - 5ms/epoch - 547us/step
Epoch 110/200
10/10 - 0s - loss: 0.4884 - accuracy: 0.7740 - 5ms/epoch - 521us/step
Epoch 111/200
10/10 - 0s - loss: 0.4938 - accuracy: 0.7774 - 5ms/epoch - 501

Epoch 198/200
10/10 - 0s - loss: 0.3381 - accuracy: 0.8288 - 6ms/epoch - 581us/step
Epoch 199/200
10/10 - 0s - loss: 0.3890 - accuracy: 0.8116 - 5ms/epoch - 528us/step
Epoch 200/200
10/10 - 0s - loss: 0.3410 - accuracy: 0.8562 - 5ms/epoch - 520us/step


In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.6954 - accuracy: 0.7653 - 41ms/epoch - 10ms/step
Loss: 0.695381224155426, Accuracy: 0.7653061151504517


# CRE Loan Group

In [35]:
# Create CRE loan dataset
CRE_df = main_df[['observation_date', 'CRE_CO', 'CRE_DELNQ', 'Rental_Vacancy_Rate', 'GDP', 'CORP_DEBT_NET_WORTH', 'Manufacturing_Confidence', 'FEDFUNDS', 'CORP_SAVINGS_LEVEL', 'UNRATE']]

CRE_bin_df = mort_df.copy()

CRE_df.columns

Index(['observation_date', 'CRE_CO', 'CRE_DELNQ', 'Rental_Vacancy_Rate', 'GDP',
       'CORP_DEBT_NET_WORTH', 'Manufacturing_Confidence', 'FEDFUNDS',
       'CORP_SAVINGS_LEVEL', 'UNRATE'],
      dtype='object')

In [36]:
# Binning the data for classification Question: 
CRE_bin_df["CRE_CO_BIN"] = pd.qcut(CRE_df['CRE_CO'],4, labels= [1, 2, 3, 4])

# Define the dependent Y variable
y = CRE_bin_df["CRE_CO_BIN"]

CRE_df = CRE_df.drop(columns=['CRE_CO','observation_date'])
CRE_df

,CRE_DELNQ,Rental_Vacancy_Rate,GDP,CORP_DEBT_NET_WORTH,Manufacturing_Confidence,FEDFUNDS,CORP_SAVINGS_LEVEL,UNRATE
0,12.08,7.5,-1.9,46.132964,96.951745,6.91,77.964,6.4
1,12.08,7.5,-1.9,46.132964,96.972896,6.25,77.964,6.6
2,12.08,7.5,-1.9,46.132964,97.223425,6.12,77.964,6.8
3,11.82,7.3,3.2,46.289579,97.678049,5.91,81.294,6.7
4,11.82,7.3,3.2,46.289579,98.292261,5.78,81.294,6.9
...,...,...,...,...,...,...,...,...
385,0.77,6.4,2.2,40.497128,98.951152,4.57,263.194,3.6
386,0.77,6.4,2.2,40.497128,98.913862,4.65,263.194,3.5
387,0.84,6.3,2.1,39.659559,98.904602,4.83,367.036,3.4
388,0.84,6.3,2.1,39.659559,98.887364,5.06,367.036,3.7


# Machine Learning: CRE Loans

In [37]:
## Splitting into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(ccard_df, y, random_state=78)

In [38]:
# Creating StandardScaler instance
scaler = MinMaxScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
print("X_train set shape: ", X_train_scaled.shape)
print("y_train set shape: ", y_train.shape)

X_train set shape:  (292, 7)
y_train set shape:  (292,)


In [40]:
y_train.values

[2, 3, 4, 2, 3, ..., 2, 1, 1, 4, 3]
Length: 292
Categories (4, int64): [1 < 2 < 3 < 4]

In [41]:
num_classes = 5
y_train = to_categorical(y_train.values, num_classes)
y_test = to_categorical(y_test.values, num_classes)
y_train[0]

array([0., 0., 1., 0., 0.], dtype=float32)

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
keras.backend.clear_session()
model = Sequential()

number_input_features = X_train_scaled.shape[1]
hidden_nodes_layers1 = 60
hidden_nodes_layers2 = 60
hidden_nodes_layers3 = 60
hidden_nodes_layers4 = 40


model.add(Dense(20, activation='relu', input_dim=number_input_features))
model.add(Dense(hidden_nodes_layers1, activation='relu'))
model.add(Dense(hidden_nodes_layers2, activation='relu'))
model.add(Dense(hidden_nodes_layers3, activation='relu'))
model.add(Dense(hidden_nodes_layers4, activation='relu'))
# model.add(Dense(hidden_nodes_layers5, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                160       
                                                                 
 dense_1 (Dense)             (None, 60)                1260      
                                                                 
 dense_2 (Dense)             (None, 60)                3660      
                                                                 
 dense_3 (Dense)             (None, 60)                3660      
                                                                 
 dense_4 (Dense)             (None, 40)                2440      
                                                                 
 dense_5 (Dense)             (None, 5)                 205       
                                                                 
Total params: 11385 (44.47 KB)
Trainable params: 11385 (

In [43]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=200,
    shuffle=True,
    verbose=2
)

Epoch 1/200
10/10 - 0s - loss: 1.5768 - accuracy: 0.2329 - 126ms/epoch - 13ms/step
Epoch 2/200
10/10 - 0s - loss: 1.5112 - accuracy: 0.2329 - 5ms/epoch - 521us/step
Epoch 3/200
10/10 - 0s - loss: 1.4450 - accuracy: 0.2329 - 5ms/epoch - 509us/step
Epoch 4/200
10/10 - 0s - loss: 1.3639 - accuracy: 0.2877 - 5ms/epoch - 469us/step
Epoch 5/200
10/10 - 0s - loss: 1.2727 - accuracy: 0.3699 - 5ms/epoch - 499us/step
Epoch 6/200
10/10 - 0s - loss: 1.1605 - accuracy: 0.5479 - 5ms/epoch - 458us/step
Epoch 7/200
10/10 - 0s - loss: 1.0419 - accuracy: 0.6096 - 6ms/epoch - 565us/step
Epoch 8/200
10/10 - 0s - loss: 0.9444 - accuracy: 0.6747 - 5ms/epoch - 471us/step
Epoch 9/200
10/10 - 0s - loss: 0.8654 - accuracy: 0.6918 - 5ms/epoch - 486us/step
Epoch 10/200
10/10 - 0s - loss: 0.8330 - accuracy: 0.6781 - 5ms/epoch - 540us/step
Epoch 11/200
10/10 - 0s - loss: 0.7464 - accuracy: 0.7192 - 5ms/epoch - 524us/step
Epoch 12/200
10/10 - 0s - loss: 0.7169 - accuracy: 0.7192 - 6ms/epoch - 552us/step
Epoch 13/200

Epoch 100/200
10/10 - 0s - loss: 0.2460 - accuracy: 0.8938 - 5ms/epoch - 518us/step
Epoch 101/200
10/10 - 0s - loss: 0.2466 - accuracy: 0.9144 - 5ms/epoch - 510us/step
Epoch 102/200
10/10 - 0s - loss: 0.2171 - accuracy: 0.9110 - 5ms/epoch - 524us/step
Epoch 103/200
10/10 - 0s - loss: 0.2027 - accuracy: 0.9349 - 5ms/epoch - 533us/step
Epoch 104/200
10/10 - 0s - loss: 0.2034 - accuracy: 0.9144 - 5ms/epoch - 517us/step
Epoch 105/200
10/10 - 0s - loss: 0.1973 - accuracy: 0.9418 - 5ms/epoch - 522us/step
Epoch 106/200
10/10 - 0s - loss: 0.1873 - accuracy: 0.9349 - 5ms/epoch - 507us/step
Epoch 107/200
10/10 - 0s - loss: 0.1809 - accuracy: 0.9384 - 5ms/epoch - 507us/step
Epoch 108/200
10/10 - 0s - loss: 0.1781 - accuracy: 0.9247 - 5ms/epoch - 518us/step
Epoch 109/200
10/10 - 0s - loss: 0.1839 - accuracy: 0.9212 - 5ms/epoch - 517us/step
Epoch 110/200
10/10 - 0s - loss: 0.1852 - accuracy: 0.9315 - 5ms/epoch - 520us/step
Epoch 111/200
10/10 - 0s - loss: 0.1866 - accuracy: 0.9384 - 5ms/epoch - 518

Epoch 198/200
10/10 - 0s - loss: 0.2905 - accuracy: 0.8973 - 5ms/epoch - 535us/step
Epoch 199/200
10/10 - 0s - loss: 0.2075 - accuracy: 0.9144 - 5ms/epoch - 508us/step
Epoch 200/200
10/10 - 0s - loss: 0.2884 - accuracy: 0.8904 - 5ms/epoch - 504us/step


In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.4286 - accuracy: 0.8367 - 40ms/epoch - 10ms/step
Loss: 0.4286101460456848, Accuracy: 0.8367347121238708
